In [1]:
from train import train_one_set
import yaml


# Test model's performance using default hyperparameters and different ratios

In [ ]:
config = yaml.load(open('config.yaml', 'r'), Loader=yaml.FullLoader)
ratios = [(0.2, 0.2), (0.3, 0.2), (0.4, 0.2), (0.5, 0.2), (0.6, 0.2)]
lst_result = []
for ratio in ratios:
    for fold in range(0, 5):
        # Print separator line for readability
        print('='*20)
        
        # Log current experiment configuration
        print(f"Running ratio {ratio} and fold {fold}")
        
        # Train model and get results for current ratio/fold combination
        # dict_cur_result contains test/validation accuracies for XGBoost, contrastive, and supervised models
        dict_cur_result = train_one_set(config, ratio, fold)
        
        # Store experiment parameters
        dict_cur_result['ratio'] = ratio  # Training/validation split ratio
        dict_cur_result['fold'] = fold    # Current cross-validation fold
        
        # Add results to master list
        lst_result.append(dict_cur_result)
        
        # Print model performance metrics
        print(f'XGBoost Test Acc: {dict_cur_result["xgb_test_acc"]}')
        print(f'Contrastive Test Acc: {dict_cur_result["constrative_test_acc"]}')
        print(f'Supervised Test Acc: {dict_cur_result["supervised_test_acc"]}')
        
        # Print separator line
        print('='*20)



In [16]:
# Initialize a dictionary to store merged results for different training/validation ratios.
# Each key is a tuple representing the ratio (train_ratio, val_ratio), and the value is another dictionary
# that holds the accumulated accuracy metrics for XGBoost, contrastive, and supervised models.
dict_merged_result = {
    (0.2, 0.2): {'xgb_test_acc': 0, 'constrative_test_acc': 0, 'supervised_test_acc': 0, 'xgb_validation_acc': 0, 'constrative_validation_acc': 0, 'supervised_validation_acc': 0},
    (0.3, 0.2): {'xgb_test_acc': 0, 'constrative_test_acc': 0, 'supervised_test_acc': 0, 'xgb_validation_acc': 0, 'constrative_validation_acc': 0, 'supervised_validation_acc': 0},
    (0.4, 0.2): {'xgb_test_acc': 0, 'constrative_test_acc': 0, 'supervised_test_acc': 0, 'xgb_validation_acc': 0, 'constrative_validation_acc': 0, 'supervised_validation_acc': 0},
    (0.5, 0.2): {'xgb_test_acc': 0, 'constrative_test_acc': 0, 'supervised_test_acc': 0, 'xgb_validation_acc': 0, 'constrative_validation_acc': 0, 'supervised_validation_acc': 0},
    (0.6, 0.2): {'xgb_test_acc': 0, 'constrative_test_acc': 0, 'supervised_test_acc': 0, 'xgb_validation_acc': 0, 'constrative_validation_acc': 0, 'supervised_validation_acc': 0},
}

# Iterate through the results collected from different experiments and accumulate the accuracy metrics
for result in lst_result:
    dict_merged_result[result['ratio']]['xgb_test_acc'] += result['xgb_test_acc']
    dict_merged_result[result['ratio']]['constrative_test_acc'] += result['constrative_test_acc'].detach().cpu().numpy()
    dict_merged_result[result['ratio']]['supervised_test_acc'] += result['supervised_test_acc'].detach().cpu().numpy()
    dict_merged_result[result['ratio']]['xgb_validation_acc'] += result['xgb_val_acc']
    dict_merged_result[result['ratio']]['constrative_validation_acc'] += result['constrative_val_acc'].detach().cpu().numpy()
    dict_merged_result[result['ratio']]['supervised_validation_acc'] += result['supervised_val_acc'].detach().cpu().numpy()

# Prepare the final results by averaging the accumulated metrics over the number of folds (5)
lst_final_result = []
for ratio in dict_merged_result:
    for key in dict_merged_result[ratio]:
        dict_merged_result[ratio][key] /= 5  # Average the results
    dict_merged_result[ratio]['ratio'] = ratio  # Add the ratio to the result dictionary
    lst_final_result.append(dict_merged_result[ratio])  # Append the result to the final list

# Convert the final results into a DataFrame and save it as a CSV file
import pandas as pd
pd.DataFrame(lst_final_result).to_csv('result.csv', index=False)

In [2]:
# Dictionary defining the hyperparameter search space:
# - lr: Learning rates to try (0.005, 0.01)
# - lams: Pairs of lambda values for loss weighting [(λ1, λ2), ...]
# - p: Pairs of probabilities for strong/weak augmentation [(p_strong, p_weak), ...]
# - hidden_dim: Hidden layer dimensions to try
hypter_params = {
    'lr': [0.005, 0.01],
    'lams': [(0.1, 0.9), (0.3, 0.7), (0.5, 0.5)],
    'p': [(0.5, 0.5), (0.7, 0.3), (0.9, 0.1)],
    'hidden_dim': [16,32, 128],
}

# Generate all possible combinations of hyperparameters using itertools.product
import itertools
lst_hyper_params = list(itertools.product(hypter_params['lr'], hypter_params['lams'], hypter_params['p'], hypter_params['hidden_dim']))
lst_result_hyper_params = []
testing_ratio = (0.2, 0.2)  # Fixed train/test split ratio

# Iterate through each hyperparameter combination
for hyper_params in lst_hyper_params:
    temp_result = []
    # Load base configuration from yaml file
    config = yaml.load(open('config.yaml', 'r'), Loader=yaml.FullLoader)
    print(f'Running hyper params: {hyper_params}')
    
    # Update configuration with current hyperparameter values:
    # - Learning rate for pretraining
    config['pretrain']['lr'] = hyper_params[0]
    # - Lambda values for loss weighting
    config['hyperparameters']['lam'] = hyper_params[1][0]
    config['hyperparameters']['lam2'] = hyper_params[1][1]
    # - Augmentation probabilities for edges and nodes
    config['augmentation']['p_edge_1'] = hyper_params[2][0]
    config['augmentation']['p_node_1'] = hyper_params[2][0]
    config['augmentation']['p_edge_2'] = hyper_params[2][1]
    config['augmentation']['p_node_2'] = hyper_params[2][1]
    # - Model architecture dimensions
    config['model']['hidden_dim'] = hyper_params[3]
    config['model']['output_dim'] = hyper_params[3]
    
    # Train model with current configuration and store results
    dict_avg_result = train_one_set(config, testing_ratio, 0)
    dict_avg_result['ratio'] = testing_ratio
    dict_avg_result['hyper_params'] = hyper_params
    lst_result_hyper_params.append(dict_avg_result)


Running hyper params: (0.005, (0.1, 0.9), (0.5, 0.5), 16)
data/noise


/mnt/raid14t/projects/ECE1508/final_project/data.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(path)
/home/dd/anaconda3/envs/pt/li

Best Val Acc: 0.5385 - Best Test Acc: 0.3247
Best Val Acc: 0.4615 - Best Test Acc: 0.2338
Running hyper params: (0.005, (0.1, 0.9), (0.5, 0.5), 32)
data/noise


/mnt/raid14t/projects/ECE1508/final_project/data.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(path)
/home/dd/anaconda3/envs/pt/li

Best Val Acc: 0.6154 - Best Test Acc: 0.2727
Best Val Acc: 0.6538 - Best Test Acc: 0.2597
Running hyper params: (0.005, (0.1, 0.9), (0.5, 0.5), 128)
data/noise


...

Best Val Acc: 0.6538 - Best Test Acc: 0.2597
Best Val Acc: 0.5385 - Best Test Acc: 0.3117
Running hyper params: (0.01, (0.5, 0.5), (0.9, 0.1), 32)
data/noise


/mnt/raid14t/projects/ECE1508/final_project/data.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(path)
/home/dd/anaconda3/envs/pt/li

Best Val Acc: 0.6538 - Best Test Acc: 0.2727
Best Val Acc: 0.5000 - Best Test Acc: 0.2727
Running hyper params: (0.01, (0.5, 0.5), (0.9, 0.1), 128)
data/noise


/mnt/raid14t/projects/ECE1508/final_project/data.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(path)
/home/dd/anaconda3/envs/pt/li

Best Val Acc: 0.6154 - Best Test Acc: 0.2208
Best Val Acc: 0.5385 - Best Test Acc: 0.2987


In [6]:
import pandas as pd
# for result in lst_result_hyper_params:
#     result['constrative_val_acc'] = result['constrative_val_acc'].detach().cpu().numpy()
#     result['constrative_test_acc'] = result['constrative_test_acc'].detach().cpu().numpy()
#     result['supervised_val_acc'] = result['supervised_val_acc'].detach().cpu().numpy()
#     result['supervised_test_acc'] = result['supervised_test_acc'].detach().cpu().numpy()

pd.DataFrame(lst_result_hyper_params).sort_values(by='constrative_test_acc', ascending=False).to_csv('result_hyper_params_mean.csv', index=False)

In [15]:
df = pd.DataFrame(lst_result_hyper_params).sort_values(by='constrative_test_acc', ascending=False)
df['train_ratio'] = df['ratio'].apply(lambda x: x[0])
df['learning_rate'] = df['hyper_params'].apply(lambda x: x[0])
df['lam'] = df['hyper_params'].apply(lambda x: x[1][0])
df['lam2'] = df['hyper_params'].apply(lambda x: x[1][1])
df['strong_P'] = df['hyper_params'].apply(lambda x: x[2][0])
df['weak_P'] = df['hyper_params'].apply(lambda x: x[2][1])
df['hidden_dim'] = df['hyper_params'].apply(lambda x: x[3])

df.drop(columns=['hyper_params', 'ratio']).to_csv('result_hyper_params_mean.csv', index=False)